In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
#Dataloader

#Path for training and testing directory
train_path='C:/Users/Dev Prajapati/Desktop/CNN/final_marathi_mask_splitdataset/train'
test_path='C:/Users/Dev Prajapati/Desktop/CNN/final_marathi_mask_splitdataset/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [6]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['a', 'aa', 'ai', 'au', 'b', 'bh', 'ch', 'cha', 'd', 'da', 'dh', 'dha', 'e', 'ee', 'f', 'g', 'gh', 'ha', 'i', 'j', 'jah', 'jaha', 'jh', 'k', 'kh', 'la', 'lla', 'ma', 'n', 'na', 'oo', 'p', 'ra', 's', 'sha', 't', 'ta', 'th', 'tha', 'u', 'va', 'ya']


In [8]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=42):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,128,128)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,128,128)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,128,128)
        self.relu1=nn.ReLU()
        #Shape= (256,12,128,128)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,64,64)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,64,64)
        self.relu2=nn.ReLU()
        #Shape= (256,12,64,64)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,64,64)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,64,64)
        self.relu3=nn.ReLU()
        #Shape= (256,32,64,64)
        
        self.fc=nn.Linear(in_features=112*112*32,out_features=num_classes)
        #self.sf=nn.Softmax(in_features=112 * 112 * 32,out_features=num_classes)
        #self.softmax=nn.Softmax()
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
            
            #Above output will be in matrix form, with shape (256,32,64,64)
            
        output=output.view(-1,32*112*112)
        
        #output=self.sf(output)
        #output=self.softmax(output)    
        output=self.fc(output)
            
        return output
            
        


In [9]:
model=ConvNet(num_classes=42).to(device)

In [10]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [11]:
num_epochs=15

In [12]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))+len(glob.glob(train_path+'/**/*.jpeg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))+len(glob.glob(test_path+'/**/*.jpeg'))

In [13]:
print(train_count,test_count)

13440 3360


In [14]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'final_mask_marathi_best_checkpoint.model')
        best_accuracy=test_accuracy
    
       


Epoch: 0 Train Loss: tensor(28.2855) Train Accuracy: 0.5196428571428572 Test Accuracy: 0.7047619047619048
Epoch: 1 Train Loss: tensor(1.2833) Train Accuracy: 0.8314732142857143 Test Accuracy: 0.8014880952380953
Epoch: 2 Train Loss: tensor(0.5079) Train Accuracy: 0.9129464285714286 Test Accuracy: 0.8407738095238095
Epoch: 3 Train Loss: tensor(0.2171) Train Accuracy: 0.9553571428571429 Test Accuracy: 0.8693452380952381
Epoch: 4 Train Loss: tensor(0.1215) Train Accuracy: 0.9716517857142857 Test Accuracy: 0.8654761904761905
Epoch: 5 Train Loss: tensor(0.0796) Train Accuracy: 0.9800595238095238 Test Accuracy: 0.8791666666666667
Epoch: 6 Train Loss: tensor(0.0703) Train Accuracy: 0.9835565476190476 Test Accuracy: 0.8821428571428571
Epoch: 7 Train Loss: tensor(0.0429) Train Accuracy: 0.9905505952380952 Test Accuracy: 0.874702380952381
Epoch: 8 Train Loss: tensor(0.0455) Train Accuracy: 0.9889136904761905 Test Accuracy: 0.8738095238095238
Epoch: 9 Train Loss: tensor(0.0459) Train Accuracy: 0.9

KeyboardInterrupt: 